In [25]:
import os
import sys
import glob
import math
import numpy as np
import time
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras.layers import Dense, Softmax
from keras import Model

In [2]:
print(tf.__version__)

2.15.0


## Test load model from h5 file

In [3]:
model = load_model("resources/pretrained_models/acdnet20_20khz_fold4.h5")

2023-12-05 13:00:10.329439: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-05 13:00:10.649254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-05 13:00:10.649573: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 1, 15109, 4)       36        
                                                                 
 batch_normalization (Batch  (None, 1, 15109, 4)       16        
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 1, 15109, 4)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 7553, 32)       640       
                                                                 
 batch_normalization_1 (Bat  (None, 1, 7553, 32)       128       
 chNormalization)                                            

In [16]:
print(model.layers[47])

## make first 10 layer frezze

In [17]:
num_classes = 10

In [14]:
for i in range(47):
    model.layers[i].trainable = False

In [15]:
for i in range(46,49):
    model.layers[i].trainable = True

In [8]:
# model.summary()

In [26]:
custom_layers = model.layers[46].output
custom_layers = Dense(num_classes)(custom_layers)
custom_layers = Softmax()(custom_layers)
# custom_layers = Dense(num_classes,activation="softmax")(custom_layers)

In [27]:
new_model = Model(inputs=model.input,outputs=custom_layers)

In [28]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 1, 15109, 4)       36        
                                                                 
 batch_normalization (Batch  (None, 1, 15109, 4)       16        
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 1, 15109, 4)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 7553, 32)       640       
                                                                 
 batch_normalization_1 (Bat  (None, 1, 7553, 32)       128       
 chNormalization)                                          

In [31]:
# print(model.weights)

In [33]:
for l in new_model.layers:
    print(f"layer : {l}\n")
    print(f"{len(l.weights)}")

layer : <keras.src.engine.input_layer.InputLayer object at 0x7f9b1c278730>

0
layer : <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f9a87aec640>

1
layer : <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7f9a87aec970>

4
layer : <keras.src.layers.activation.relu.ReLU object at 0x7f9a87aecfa0>

0
layer : <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f9a87aed540>

1
layer : <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7f9a87aed660>

4
layer : <keras.src.layers.activation.relu.ReLU object at 0x7f9a87aedc30>

0
layer : <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f9a87aee320>

0
layer : <keras.src.layers.reshaping.permute.Permute object at 0x7f9a87aee740>

0
layer : <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f9a87aee770>

1
layer : <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7f9a87aeea40>

4
layer : <keras.src.layers.